## Black Hole Evolution Dataset (TNG100)
This notebook builds a dataset for modeling the evolution of black holes using the IllustrisTNG100 simulation (snapshots 18–33). The dataset will be used to train an LSTM to predict future black hole properties based on their past evolution.

### 1. Environment Setup
---
Import necessary libraries and configure global settings for reproducibility.


In [1]:
import requests
import numpy as np
import torch
import random

random.seed(42)  # Ensures reproducible random sampling later

print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")

NumPy version: 1.24.3
PyTorch version: 2.0.1+cpu


### 2. Data Access & Preprocessing
---

#### 2.1 Load Subhalo Catalog (Snapshot 33)
---
We begin by selecting black-hole-hosting subhalos at snapshot 33 (z ≈ 0, present day).

In [2]:
import illustris_python as il

basePath = "/home/tnguser/sims.TNG/TNG100-1/output" # Adjust based on Environment

subhalos = il.groupcat.loadSubhalos(
    basePath, 
    33, 
    fields=['SubhaloBHMass', 'SubhaloMassType']
)

bh_mass = subhalos['SubhaloBHMass']
stellar_mass = subhalos['SubhaloMassType'][:, 4]  # Type 4 = stellar component

bh_mask = bh_mass > 0
print(f"Total subhalos with black holes: {bh_mask.sum()}")

Total subhalos with black holes: 29415


#### 2.2 Extract Black Hole Evolutionary Histories
---
Trace each black hole’s most bound progenitor branch through merger trees and store time-ordered properties across snapshots 18-33.


In [3]:
tree_base = f"{basePath}/postprocessing/trees/sublink"

full_histories = {}
bh_list = [i for i, has_bh in enumerate(bh_mask) if has_bh]
required_snaps = set(range(18, 34))

for count, sub_id in enumerate(bh_list, start=1):
    try:
        tree = il.sublink.loadTree(
            basePath,
            33,
            sub_id,
            fields=[
                'SubhaloID',
                'SnapNum',
                'SubhaloBHMass',
                'SubhaloBHMdot',
                'SubhaloMassType',
                'SubhaloSFR',
                'SubhaloVelDisp'
            ],
            onlyMPB=True
        )

        mask = (tree['SnapNum'] <= 32) & (tree['SnapNum'] >= 18)
        snaps = set(tree['SnapNum'][mask])

        # Keep only if ≥90% snapshot coverage
        if len(snaps & required_snaps) >= int(0.9 * len(required_snaps)):
            sorted_idx = np.argsort(tree['SnapNum'][mask])
            full_histories[sub_id] = {
                "snap_nums": tree['SnapNum'][mask][sorted_idx].tolist(),
                "bh_mass": tree['SubhaloBHMass'][mask][sorted_idx].tolist(),
                "bh_accretion": tree['SubhaloBHMdot'][mask][sorted_idx].tolist(),
                "stellar_mass": tree['SubhaloMassType'][mask][sorted_idx, 4].tolist(),
                "halo_mass": tree['SubhaloMassType'][mask][sorted_idx].sum(axis=1).tolist(),
                "sfr": tree['SubhaloSFR'][mask][sorted_idx].tolist(),
                "vel_dispersion": tree['SubhaloVelDisp'][mask][sorted_idx].tolist()
            }
    except:
        continue

    if count % 5000 == 0:
        print(f"Checked {count}/{len(bh_list)} subhalos...", flush=True)

print(f"Black holes with ≥90% snapshot coverage: {len(full_histories)}")


Warning, empty return. Subhalo [18022] at snapNum [33] not in tree.
Warning, empty return. Subhalo [19588] at snapNum [33] not in tree.
Warning, empty return. Subhalo [39449] at snapNum [33] not in tree.
Warning, empty return. Subhalo [39547] at snapNum [33] not in tree.
Warning, empty return. Subhalo [138375] at snapNum [33] not in tree.
Warning, empty return. Subhalo [147013] at snapNum [33] not in tree.
Checked 5000/29415 subhalos...
Checked 10000/29415 subhalos...
Checked 15000/29415 subhalos...
Checked 20000/29415 subhalos...
Checked 25000/29415 subhalos...
Black holes with ≥90% snapshot coverage: 29232


#### 2.3 Sample and Verify Black Holes
---
Randomly select 2,500 black holes with ≥90% snapshot coverage 
and inspect one example to confirm that time-series properties 
were extracted correctly.

In [4]:
sampled_ids = random.sample(list(full_histories.keys()), 2500)
print(f"Sampled {len(sampled_ids)} black holes.")


Sampled 2500 black holes.


In [5]:
# Verification
check_id = random.choice(sampled_ids)
print(f"Inspecting black hole ID: {check_id}")
print("Snapshots:", full_histories[check_id]["snap_nums"])
print("BH Mass Sequence:", full_histories[check_id]["bh_mass"])


Inspecting black hole ID: 163461
Snapshots: [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
BH Mass Sequence: [9.446660260437056e-05, 9.979942115023732e-05, 0.00010838297748705372, 0.00012950033124070615, 0.00015543214976787567, 0.00019729389168787748, 0.0002733152941800654, 0.0005854691262356937, 0.0008536182576790452, 0.0012909594224765897, 0.0018033889355137944, 0.004496393725275993, 0.010067204013466835, 0.012057174928486347, 0.013648551888763905]


#### 2.4 Prepare Dataset for Machine Learning
---
Structure the extracted properties into a consistent time-series format and save as a CSV for future training.

In [6]:
import pandas as pd
import os

# Ensure project-specific data directory exists (relative to notebooks folder)
os.makedirs("../data", exist_ok=True)

data_rows = []
for sub_id in sampled_ids:
    bh = full_histories[sub_id]
    row = {"subhalo_id": sub_id}
    for i, snap in enumerate(bh["snap_nums"]):
        row[f"bh_mass_snap{snap}"] = bh["bh_mass"][i]
        row[f"bh_acc_snap{snap}"] = bh["bh_accretion"][i]
        row[f"stellar_mass_snap{snap}"] = bh["stellar_mass"][i]
        row[f"sfr_snap{snap}"] = bh["sfr"][i]
        row[f"halo_mass_snap{snap}"] = bh["halo_mass"][i]
        row[f"vel_disp_snap{snap}"] = bh["vel_dispersion"][i]
    data_rows.append(row)

ml_dataset = pd.DataFrame(data_rows)

# Save CSV in black_hole_evolution/data
csv_path = "../data/black_hole_evolution_tng100.csv"
ml_dataset.to_csv(csv_path, index=False)

print(f"Dataset saved: {csv_path}")
print(f"Shape: {ml_dataset.shape[0]} black holes, {ml_dataset.shape[1]} features")
ml_dataset.head(3)


Dataset saved: ../data/black_hole_evolution_tng100.csv
Shape: 2500 black holes, 91 features


,subhalo_id,bh_mass_snap18,bh_acc_snap18,stellar_mass_snap18,sfr_snap18,halo_mass_snap18,vel_disp_snap18,bh_mass_snap19,bh_acc_snap19,stellar_mass_snap19,...,stellar_mass_snap31,sfr_snap31,halo_mass_snap31,vel_disp_snap31,bh_mass_snap32,bh_acc_snap32,stellar_mass_snap32,sfr_snap32,halo_mass_snap32,vel_disp_snap32
0,386132,0.0,0.0,0.000905,0.060235,0.789287,39.962952,0.0,0.0,0.001301,...,0.022721,0.486914,5.505792,55.783905,0.000084,0.000005,0.028393,0.423227,6.073032,55.394325
1,130735,0.0,0.0,0.002902,0.103475,1.530316,47.719479,0.0,0.0,0.003578,...,0.043805,0.301828,3.783332,55.236423,0.000118,0.000025,0.045069,0.325648,3.724669,53.772259
2,37402,0.0,0.0,0.001395,0.080354,0.221330,34.087959,0.0,0.0,0.003692,...,0.053857,1.459296,1.262915,55.320915,0.000110,0.000041,0.065400,1.443447,1.258591,50.473103
